In [ ]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Create multiIndex

### Implicit MultiIndex constructors

In [ ]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['col1', 'col2'])
df

In [ ]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}

ds = pd.Series(data)
ds

###  Explicit MultiIndex constructors¶

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

### Level names

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=pd.MultiIndex.from_tuples(index))
pop

### With more involved datasets, this can be a useful way to keep track of the meaning of various index values.

In [ ]:
pop.index.names = ['state', 'year']
pop

In [ ]:
df = pd.DataFrame({'a': range(7), 
                'b': range(7, 0, -1),
                'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                'd': [0, 1, 2, 0, 1, 2, 3]}
              )
df

In [ ]:
df.set_index(['c','d'])

In [ ]:
pop.reset_index(name='population')

## Indexing and Slicing a MultiIndex

In [ ]:
index = [('California', 2000), ('California', 2010),
        ('New York', 2000), ('New York', 2010),
          ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
              18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=pd.MultiIndex.from_tuples(index))
pop

In [ ]:
pop

In [ ]:
pop['California', 2000]

In [ ]:
pop['California']

In [ ]:
pop.loc['California':'New York']

### Partial slicing is available as well, as long as the `MultiIndex` is sorted

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('Texas', 2000), ('Texas', 2010),
        ('New York', 2000), ('New York', 2010)]
populations = [33871648, 37253956,
               20851820, 25145561,
              18976457, 19378102]
pop2 = pd.Series(populations, index=pd.MultiIndex.from_tuples(index))
pop2

In [ ]:
try:
    pop2.loc['California':'New York']
except KeyError as e:
    print(type(e))
    print(e)a

### Pandas provides a number of convenience routines to perform this type of sorting; examples are the `sort_index()` and `sortlevel()` methods of the `DataFrame`.

In [ ]:
pop2.index

In [ ]:
pop2.sort_index()['California':'New York']

In [ ]:
pop2.sort_index()['California':'New York']

In [ ]:
pop2[:,2000]

In [ ]:
pop[['California', 'Texas']]

In [ ]:
pop.unstack(level=0)
pop.unstack(level=1)

## Index setting and resetting

In [ ]:
pop.index.names = ['state','year']
pop

In [ ]:
pop_flat = pop.reset_index(name='population')
pop_flat

### Often when working with data in the real world, the raw input data looks like this and it's useful to build a `MultiIndex` from the column values.

In [ ]:
pop_flat.set_index(['state', 'year'])

## MultiIndex for columns

In [ ]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

In [ ]:
health_data.iloc[0:2,3:5]

In [ ]:
health_data.loc[2013,['Guido','Sue']]

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

[multiIndex](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html)

### Using slicers

In [ ]:
health_data.loc[(slice(None),1), ('Bob', 'HR')]

In [ ]:
idx = pd.IndexSlice

In [ ]:
health_data.loc[idx[:,1], idx['Bob','HR']]

In [ ]:
idx[:,1]

## Data Aggregation

In [ ]:
health_data

In [ ]:
data_mean = health_data.mean(level='year')
data_mean

In [ ]:
health_data.mean(axis=1, level='type')

### How to calculate the mean of each `type` for each `year`

In [364]:
health_data.unstack('year')

subject   Bob                   Guido                     Sue              \
type       HR        Temp          HR        Temp          HR        Temp   
year     2013  2014  2013  2014  2013  2014  2013  2014  2013  2014  2013   
visit                                                                       
1        56.0  39.0  37.6  37.3  39.0  34.0  37.9  38.0  26.0  23.0  36.2   
2        15.0  47.0  38.5  37.0  55.0  31.0  37.6  37.4  42.0  32.0  35.9   

subject        
type           
year     2014  
visit          
1        36.1  
2        37.1

In [365]:
health_data.stack().stack().groupby(['year', 'type']).mean()

year  type
2013  HR      38.833333
      Temp    37.283333
2014  HR      34.333333
      Temp    37.150000
dtype: float64

## Demo

In [329]:
df = pd.read_csv('data.csv', parse_dates=['Date'])
df.sample(5)

,Date,Store,Category,Subcategory,UPC EAN,Description,Dollars,Units
49,2018-07-16,Store 3,Beer,Malts,71250000140,Colt 45 - Malt Liquor - 6 Pack,108.48,12
70,2018-07-13,Store 1,Wine,Red,492130008399,Yellow Tail - Merlot - 750 mL Bottle,38.60,5
25,2018-07-16,Store 1,Beer,Lagers,702770081011,Omission - Gluten Free Lager - 6 Pack,174.72,14
67,2018-07-14,Store 3,Wine,Red,89744402585,Trapiche - Malbec - 750 mL Bottle,20.15,5
118,2018-07-15,Store 3,Alcohol,Liquor,674545000001,Don Julio - Tequila Blanco - 750 mL Bottle,430.50,14


In [330]:
df.Date[0]

Timestamp('2018-07-11 00:00:00')

In [331]:
df.set_index(['Date', 'Store', 'Category', 'Subcategory', 'Description'], inplace=True)
df.head(5)

UPC EAN  \
Date       Store   Category Subcategory Description                                         
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   
2018-07-12 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   

                                                                             Dollars  \
Date       Store   Category Subcategory Description                                    
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack    67.36   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack    80.01   
2018-07-12 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack    78.30   

                                                                             Units  
Date       Store   Category Subcategory Description                                 
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack      8  
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack      9  
2018-07-12 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack     10

In [332]:
df.set_index('UPC EAN', append=True, inplace=True)
df.head()

Dollars  \
Date       Store   Category Subcategory Description                         UPC EAN                 
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112    35.68   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112   166.74   
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112    67.36   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112    80.01   
2018-07-12 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112    78.30   

                                                                                          Units  
Date       Store   Category Subcategory Description                         UPC EAN              
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112      4  
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112     14  
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112      8  
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112      9  
2018-07-12 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112     10

That's almost right, but we'd actually like 'Description' to show up after 'UPC EAN'. 

In [333]:
df.reorder_levels(order=['Date', 'Store', 'Category',
                         'Subcategory', 'UPC EAN', 'Description']).head(3)

Dollars  \
Date       Store   Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                          Units  
Date       Store   Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

In [334]:
df.swaplevel('Description', 'UPC EAN').head(3)

Dollars  \
Date       Store   Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                          Units  
Date       Store   Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

In [335]:
# A handy function to keep around for projects
def add_constant_index_level(df: pd.DataFrame, value, level_name: str):
    return pd.concat([df], keys=[value], names=[level_name])

In [336]:
df = add_constant_index_level(df, "Booooze", "Department")
df = df.reorder_levels(order=['Date', 'Store', 'Department', 'Category', 'Subcategory', 'UPC EAN', 'Description'])
df.head(3)

Dollars  \
Date       Store   Department Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                                     Units  
Date       Store   Department Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

In [337]:
df.index.get_level_values('Subcategory').unique()

Index(['Ales', 'Lagers', 'Stouts', 'Malts', 'Red', 'White', 'Rose', 'Liqour',
       'Liquor'],
      dtype='object', name='Subcategory')

In [338]:
df.sort_index(inplace=True)

In [339]:
idx = pd.IndexSlice

In [340]:
mask = df.index.get_level_values('Subcategory') == 'Liqour'
mask

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False,

In [311]:
df.loc[mask,'Dollars']

Date        Store    Department  Category  Subcategory  UPC EAN      Description                                
2018-07-10  Store 2  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    231.30
            Store 3  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    128.30
2018-07-11  Store 2  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    248.80
            Store 3  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    296.01
2018-07-12  Store 2  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    279.36
            Store 3  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    228.08
2018-07-15  Store 2  Booooze     Alcohol   Liqour       80480280024  Grey Goose - Imported Vodka - 750 mL Bottle    121.20
            Store 3  Boooo

In [315]:
df.loc(axis=0)[:,:,:,:,'Liqour',:,:]

KeyError: 'Liqour'

In [352]:
# method one
df.rename(index={'Liqour': 'Liquor'}).index.get_level_values('Subcategory').unique()

Index(['Lagers', 'Red', 'Rose', 'Liquor', 'Ales', 'Stouts', 'White', 'Malts'], dtype='object', name='Subcategory')

In [349]:
#method 2
df_temp = df.reset_index(level='Subcategory', drop=False)

In [350]:
df_temp.loc[df_temp.Subcategory=='Liqour', ['Subcategory']] = 'Liquor'

In [351]:
df_temp.set_index('Subcategory', append=True, inplace=True)
df_temp.index.get_level_values('Subcategory').unique()

Index(['Lagers', 'Red', 'Rose', 'Liquor', 'Ales', 'Stouts', 'White', 'Malts'], dtype='object', name='Subcategory')

### Rename index levels

In [355]:
temp_df = df.copy()
temp_df.index = df.index.set_names('Desc.', level='Description')
temp_df.head(3)

Dollars  \
Date       Store   Department Category Subcategory UPC EAN      Desc.                                            
2018-07-10 Store 1 Booooze    Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack   137.67   
                              Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle        56.88   
                                                   492130008399 Yellow Tail - Merlot - 750 mL Bottle     46.30   

                                                                                                       Units  
Date       Store   Department Category Subcategory UPC EAN      Desc.                                         
2018-07-10 Store 1 Booooze    Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack     13  
                              Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle          8  
                                                   492130008399 Yellow Tail - Merlot - 750 mL Bottle      10

### [A great resource](https://github.com/ZaxR/pandas_multiindex_tutorial/blob/master/Pandas%20MultiIndex%20Tutorial.ipynb)

In [359]:
# We can set a MultiIndex while reading a csv by referencing columns to be used in the index by number
pd.read_csv("data.csv", index_col=[0, 1, 2, 3, 4, 5], 
            skipinitialspace=True, 
            parse_dates=['Date']).head(3)

Dollars  \
Date       Store   Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                          Units  
Date       Store   Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

###  unstack(). This function "pivots" an index level to a new level of column labels whose inner-most level consists of the pivoted index labels.

In [366]:
multi_col_lvl_df = df.unstack('Store')
multi_col_lvl_df

Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN   
...                                                                                                           ...   
2018-07-16 Booooze    Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              174.72   
                               Malts       71250000140  Colt 45 - Malt Liquor - 6 Pack                        NaN   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                  126.14   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                68.46   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle           NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              81.76   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   89.76   
...                                                                                                           ...   
2018-07-16 Booooze    Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                               Malts       71250000140  Colt 45 - Malt Liquor - 6 Pack                      47.55   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                  NaN   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle         82.80   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              62.48   
         

In [379]:
multi_col_lvl_df.to_csv('multi_col_lvl_output.csv')

# Reading it back in requires the header parameter
read_multi_df = pd.read_csv('multi_col_lvl_output.csv', header=[0, 1], index_col=[0, 1, 2, 3, 4, 5],
                            skipinitialspace=True, parse_dates=[0]).head(3)

read_multi_df

Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   

                                                                                                            Units  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           9.0   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     5.0   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            6.0   

                                                                                                                   
Store                                                                                                     Store 3  
Date       Department Category Subcategory UPC EAN      Description                                                
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           5.0  
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     8.0  
           

In [380]:
# A function to check our index level dtypes to aid this example
def index_level_dtypes(df):
    return [f"{n}: {df.index.get_level_values(n).dtype}"
            for n in (df.index.names)]

index_level_dtypes(read_multi_df)

['Date: datetime64[ns]',
 'Department: object',
 'Category: object',
 'Subcategory: object',
 'UPC EAN: int64',
 'Description: object']

In [381]:
read_multi_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3 entries, (2018-07-10 00:00:00, Booooze, Alcohol, Liqour, 80480280024, Grey Goose - Imported Vodka - 750 mL Bottle) to (2018-07-10 00:00:00, Booooze, Alcohol, Liquor, 674545000001, Don Julio - Tequila Blanco - 750 mL Bottle)
Data columns (total 6 columns):
(Dollars, Store 1)    0 non-null float64
(Dollars, Store 2)    3 non-null float64
(Dollars, Store 3)    3 non-null float64
(Units, Store 1)      0 non-null float64
(Units, Store 2)      3 non-null float64
(Units, Store 3)      3 non-null float64
dtypes: float64(6)
memory usage: 875.0+ bytes


In [382]:
dollars_per_unit = multi_col_lvl_df['Dollars'] / multi_col_lvl_df['Units']
dollars_per_unit.sample(10)

,,,,,Store,Store 1,Store 2,Store 3
Date,Department,Category,Subcategory,UPC EAN,Description,,,
2018-07-10,Booooze,Alcohol,Liquor,80432400630,Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle,NaN,34.40,34.85
2018-07-13,Booooze,Wine,Rose,85000024218,Dark Horse - S. Blanc Rose - 750 mL Bottle,13.16,14.23,NaN
2018-07-10,Booooze,Beer,Stouts,83783575217,Sierra Nevada Brewing Co. - Stout - 6 Pack,NaN,11.20,11.99
2018-07-11,Booooze,Wine,White,81248501095,Curious Beasts - Chardonnay - 750 mL Bottle,NaN,21.56,19.04
2018-07-16,Booooze,Wine,Red,492130008399,Yellow Tail - Merlot - 750 mL Bottle,4.89,NaN,7.76
2018-07-14,Booooze,Beer,Lagers,702770081011,Omission - Gluten Free Lager - 6 Pack,10.63,NaN,10.72
2018-07-15,Booooze,Beer,Ales,702770082018,Omission Pale Ale - Gluten Free Pale Ale - 6 Pack,NaN,13.35,13.34
2018-07-10,Booooze,Wine,Red,89744402585,Trapiche - Malbec - 750 mL Bottle,7.11,NaN,6.30
2018-07-15,Booooze,Alcohol,Liqour,80480280024,Grey Goose - Imported Vodka - 750 mL Bottle,NaN,24.24,23.17


In [383]:
dollars_per_unit.columns

Index(['Store 1', 'Store 2', 'Store 3'], dtype='object', name='Store')

In [384]:
# Add a column level for our new measure
dollars_per_unit.columns = pd.MultiIndex.from_product([['Dollars per Unit'], dollars_per_unit.columns])

# Concat it with our original data
pd.concat([multi_col_lvl_df, dollars_per_unit], axis='columns').head(3)

Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   

                                                                                                            Units  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           9.0   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     5.0   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            6.0   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           5.0   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     8.0   
      